In [13]:
import tensorflow as tf
import keras
import chess
import chess.pgn
import chess.engine
import numpy as np
import random

In [78]:
def read(pgn, max_games = 1000):
    games = []
    with open(pgn,'r+') as g:
        for i in range(max_games):
            gm = chess.pgn.read_game(g)
            if (gm == None):
                break
            else:
                games.append(gm)
    return games
def random_game_node(game):
    boards = list(map(chess.pgn.GameNode.board, game.mainline()))
    return random.choice(boards)
def pick_positions(games):
    return list(map(random_game_node, games))
def tensorize(b):
    t = np.zeros((18, 8,8))
    temp = np.asarray(list(map(list, str(b).replace(' ','').split('\n'))))
    pieces = list('KQRBNP.kqrbnp')
    for i in range(len(pieces)):
        piece = pieces[i]
        is_piece = temp == [piece] * 8
        t[i] = np.asarray(is_piece)
    t[13] = np.ones(t[13].shape) if (b.turn == chess.WHITE) else t[13]
    t[14] = np.ones(t[14].shape) if (b.has_queenside_castling_rights(chess.WHITE)) else t[14]
    t[15] = np.ones(t[15].shape) if (b.has_queenside_castling_rights(chess.BLACK)) else t[15]
    t[16] = np.ones(t[16].shape) if (b.has_kingside_castling_rights(chess.WHITE)) else t[16]    
    t[17] = np.ones(t[17].shape) if (b.has_kingside_castling_rights(chess.BLACK)) else t[17]
    return t
def tensorize_boards(boards):
    return list(map(tensorize, boards))
def eval_with_engine(engine):    
    return (lambda board: (engine.analyse(board, chess.engine.Limit(time=0.100)))['score'].white().score(mate_score=10000))
def get_evals(boards, engine):
    return list(map(eval_with_engine(engine), boards))
def get_data(pgn='2013-01-games.pgn', engine_path = './stockfish.exe'):
    games = read(pgn)
    boards = pick_positions(games)
    engine =  chess.engine.SimpleEngine.popen_uci(engine_path)
    x = np.asarray(tensorize_boards(boards))
    y = np.asarray(get_evals(boards, engine))
    engine.quit()
    return x,y/100.0

In [113]:
x,y = get_data()
x,y

(array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 1.]],
 
         ...,
 
         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          ...,
          [1., 1., 1., ..., 1., 1., 1.],
          [1.

In [114]:
np.save('x.npy',x)
np.save('y.npy',y)

In [115]:
x = np.load('x.npy')
x = x.reshape((1000,8,8,18))

In [116]:
y = np.load('y.npy')

In [118]:
x_train, y_train, x_val, y_val, x_test, y_test = x[:700], y[:700], x[700:850], y[700:850], x[850:], y[850:]

In [119]:
from keras import layers, models

In [183]:
model = models.Sequential()
model.add(layers.Flatten(input_shape = (8,8,18)))
model.add(layers.Dense(1))

In [185]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_15 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 1153      
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


In [186]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [187]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=1000, verbose=1)

Train on 700 samples, validate on 150 samples
Epoch 1/1000
700/700 [==============================] - 1s 1ms/step - loss: 544.4508 - mean_absolute_error: 8.8842 - val_loss: 546.5228 - val_mean_absolute_error: 8.7298
Epoch 2/1000
700/700 [==============================] - 0s 82us/step - loss: 542.4828 - mean_absolute_error: 8.7893 - val_loss: 546.4429 - val_mean_absolute_error: 8.7787
Epoch 3/1000
700/700 [==============================] - 0s 77us/step - loss: 541.0154 - mean_absolute_error: 8.8496 - val_loss: 547.1631 - val_mean_absolute_error: 8.7420
Epoch 4/1000
700/700 [==============================] - 0s 83us/step - loss: 539.7718 - mean_absolute_error: 8.7868 - val_loss: 546.9927 - val_mean_absolute_error: 8.8025
Epoch 5/1000
700/700 [==============================] - 0s 75us/step - loss: 538.4240 - mean_absolute_error: 8.8265 - val_loss: 547.0368 - val_mean_absolute_error: 8.8227
Epoch 6/1000
700/700 [==============================] - 0s 76us/step - loss: 537.1891 - mean_absolut

700/700 [==============================] - 0s 76us/step - loss: 503.2902 - mean_absolute_error: 8.7573 - val_loss: 553.7939 - val_mean_absolute_error: 9.1654
Epoch 49/1000
700/700 [==============================] - 0s 70us/step - loss: 502.8667 - mean_absolute_error: 8.7522 - val_loss: 554.0962 - val_mean_absolute_error: 9.1686
Epoch 50/1000
700/700 [==============================] - 0s 76us/step - loss: 502.0214 - mean_absolute_error: 8.7549 - val_loss: 554.3257 - val_mean_absolute_error: 9.1514
Epoch 51/1000
700/700 [==============================] - 0s 84us/step - loss: 501.5187 - mean_absolute_error: 8.7413 - val_loss: 554.2112 - val_mean_absolute_error: 9.1591
Epoch 52/1000
700/700 [==============================] - 0s 82us/step - loss: 500.9102 - mean_absolute_error: 8.7666 - val_loss: 554.5864 - val_mean_absolute_error: 9.1651
Epoch 53/1000
700/700 [==============================] - 0s 86us/step - loss: 500.0991 - mean_absolute_error: 8.7298 - val_loss: 554.9851 - val_mean_absol

Epoch 96/1000
700/700 [==============================] - 0s 86us/step - loss: 476.3503 - mean_absolute_error: 8.5963 - val_loss: 558.1075 - val_mean_absolute_error: 9.4696
Epoch 97/1000
700/700 [==============================] - 0s 119us/step - loss: 475.9061 - mean_absolute_error: 8.6226 - val_loss: 559.0449 - val_mean_absolute_error: 9.3583
Epoch 98/1000
700/700 [==============================] - 0s 214us/step - loss: 475.4693 - mean_absolute_error: 8.6068 - val_loss: 558.8432 - val_mean_absolute_error: 9.3717
Epoch 99/1000
700/700 [==============================] - 0s 126us/step - loss: 475.1482 - mean_absolute_error: 8.5916 - val_loss: 558.8686 - val_mean_absolute_error: 9.3776
Epoch 100/1000
700/700 [==============================] - 0s 82us/step - loss: 474.4757 - mean_absolute_error: 8.6137 - val_loss: 559.1944 - val_mean_absolute_error: 9.3671
Epoch 101/1000
700/700 [==============================] - 0s 87us/step - loss: 474.3052 - mean_absolute_error: 8.6026 - val_loss: 559.34

700/700 [==============================] - 0s 70us/step - loss: 455.5891 - mean_absolute_error: 8.5862 - val_loss: 564.5977 - val_mean_absolute_error: 9.6649
Epoch 144/1000
700/700 [==============================] - 0s 68us/step - loss: 455.2198 - mean_absolute_error: 8.5430 - val_loss: 564.5979 - val_mean_absolute_error: 9.6616
Epoch 145/1000
700/700 [==============================] - 0s 70us/step - loss: 454.7796 - mean_absolute_error: 8.5470 - val_loss: 564.4600 - val_mean_absolute_error: 9.6692
Epoch 146/1000
700/700 [==============================] - 0s 82us/step - loss: 454.4509 - mean_absolute_error: 8.5446 - val_loss: 564.7429 - val_mean_absolute_error: 9.6743
Epoch 147/1000
700/700 [==============================] - 0s 71us/step - loss: 453.8493 - mean_absolute_error: 8.5465 - val_loss: 564.4978 - val_mean_absolute_error: 9.7057
Epoch 148/1000
700/700 [==============================] - 0s 86us/step - loss: 453.4886 - mean_absolute_error: 8.5386 - val_loss: 565.1007 - val_mean_

700/700 [==============================] - 0s 70us/step - loss: 437.9788 - mean_absolute_error: 8.5372 - val_loss: 569.4783 - val_mean_absolute_error: 9.9251
Epoch 191/1000
700/700 [==============================] - 0s 74us/step - loss: 437.5263 - mean_absolute_error: 8.5416 - val_loss: 569.8735 - val_mean_absolute_error: 9.9386
Epoch 192/1000
700/700 [==============================] - 0s 84us/step - loss: 437.2503 - mean_absolute_error: 8.5537 - val_loss: 569.8776 - val_mean_absolute_error: 9.9541
Epoch 193/1000
700/700 [==============================] - 0s 77us/step - loss: 436.8150 - mean_absolute_error: 8.5351 - val_loss: 570.0797 - val_mean_absolute_error: 9.9642
Epoch 194/1000
700/700 [==============================] - 0s 80us/step - loss: 436.6204 - mean_absolute_error: 8.5299 - val_loss: 570.3401 - val_mean_absolute_error: 9.9520
Epoch 195/1000
700/700 [==============================] - 0s 75us/step - loss: 436.0834 - mean_absolute_error: 8.5511 - val_loss: 570.3791 - val_mean_

...

Epoch 799/1000
700/700 [==============================] - 0s 76us/step - loss: 314.0436 - mean_absolute_error: 8.2965 - val_loss: 614.4651 - val_mean_absolute_error: 11.9399
Epoch 800/1000
700/700 [==============================] - 0s 95us/step - loss: 313.8259 - mean_absolute_error: 8.3335 - val_loss: 615.0795 - val_mean_absolute_error: 11.8878
Epoch 801/1000
700/700 [==============================] - 0s 83us/step - loss: 313.8994 - mean_absolute_error: 8.3002 - val_loss: 615.0047 - val_mean_absolute_error: 11.8949
Epoch 802/1000
700/700 [==============================] - 0s 86us/step - loss: 313.8284 - mean_absolute_error: 8.3105 - val_loss: 614.8595 - val_mean_absolute_error: 11.8977
Epoch 803/1000
700/700 [==============================] - 0s 82us/step - loss: 313.6322 - mean_absolute_error: 8.3026 - val_loss: 614.5215 - val_mean_absolute_error: 11.9625
Epoch 804/1000
700/700 [==============================] - 0s 80us/step - loss: 313.6648 - mean_absolute_error: 8.3111 - val_loss: 

700/700 [==============================] - 0s 69us/step - loss: 307.9895 - mean_absolute_error: 8.2766 - val_loss: 617.1070 - val_mean_absolute_error: 11.9972
Epoch 847/1000
700/700 [==============================] - 0s 70us/step - loss: 307.7578 - mean_absolute_error: 8.2625 - val_loss: 616.8868 - val_mean_absolute_error: 12.0476
Epoch 848/1000
700/700 [==============================] - 0s 94us/step - loss: 307.7288 - mean_absolute_error: 8.3000 - val_loss: 616.9959 - val_mean_absolute_error: 12.0501
Epoch 849/1000
700/700 [==============================] - 0s 90us/step - loss: 307.6964 - mean_absolute_error: 8.2854 - val_loss: 616.9994 - val_mean_absolute_error: 12.0377
Epoch 850/1000
700/700 [==============================] - 0s 83us/step - loss: 307.3202 - mean_absolute_error: 8.2781 - val_loss: 617.6147 - val_mean_absolute_error: 11.9904
Epoch 851/1000
700/700 [==============================] - 0s 84us/step - loss: 307.3519 - mean_absolute_error: 8.2815 - val_loss: 617.3987 - val_

700/700 [==============================] - 0s 77us/step - loss: 302.0714 - mean_absolute_error: 8.2470 - val_loss: 619.2999 - val_mean_absolute_error: 12.1063
Epoch 894/1000
700/700 [==============================] - 0s 78us/step - loss: 301.9236 - mean_absolute_error: 8.2562 - val_loss: 619.5400 - val_mean_absolute_error: 12.1029
Epoch 895/1000
700/700 [==============================] - 0s 83us/step - loss: 301.8422 - mean_absolute_error: 8.2343 - val_loss: 619.8067 - val_mean_absolute_error: 12.0888
Epoch 896/1000
700/700 [==============================] - 0s 83us/step - loss: 301.7647 - mean_absolute_error: 8.2341 - val_loss: 619.6600 - val_mean_absolute_error: 12.1098
Epoch 897/1000
700/700 [==============================] - 0s 85us/step - loss: 301.5508 - mean_absolute_error: 8.2416 - val_loss: 619.5789 - val_mean_absolute_error: 12.1126
Epoch 898/1000
700/700 [==============================] - 0s 85us/step - loss: 301.4376 - mean_absolute_error: 8.2300 - val_loss: 619.5785 - val_

700/700 [==============================] - 0s 75us/step - loss: 296.4661 - mean_absolute_error: 8.2068 - val_loss: 621.2446 - val_mean_absolute_error: 12.1679
Epoch 941/1000
700/700 [==============================] - 0s 75us/step - loss: 296.2763 - mean_absolute_error: 8.1927 - val_loss: 621.7704 - val_mean_absolute_error: 12.1504
Epoch 942/1000
 32/700 [>.............................] - ETA: 0s - loss: 352.8774 - mean_absolute_error: 9.9492

KeyboardInterrupt: 

In [ ]:
mae_history = history.history['val_mean_absolute_error']
all_mae_histories.append(mae_history)